In [1]:
import os
import pandas as pd
import numpy as np
import torch.nn as nn
import torch
import lightning.pytorch as pl
from pytorch_lightning.loggers import WandbLogger
import segmentation_models_pytorch as smp
from torch.optim.lr_scheduler import CosineAnnealingLR, ReduceLROnPlateau
from torch.optim import AdamW
import torch.nn as nn
from lightning.pytorch.callbacks import ProgressBar
from torchmetrics.functional import dice, f1_score, jaccard_index
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import torch.nn.functional as F
import torchvision.models as models
from sklearn.metrics import accuracy_score, roc_auc_score
# from torchvision.models import resnet50, ResNet50_Weights

# config
config = {
    "wandb": True,
    'competition'   : 'rsna-atd' ,
    '_wandb_kernel' : 'johnnyhyl',
    "data_path": "../",
    "model": {
        "encoder_name": 'se-resnext26d',
        "loss_smooth": 1.0,
        "loss": nn.BCEWithLogitsLoss(),
        "optimizer_params": {"lr": 0.01, "weight_decay": 0.0},
        "scheduler": {
            "name": "CosineAnnealingLR",
            "params": {
                "CosineAnnealingLR": {"T_max": 20, "eta_min": 1e-06, "last_epoch": -1},
                "ReduceLROnPlateau": {
                    "factor": 0.316,
                    "mode": "min",
                    "patience": 3,
                    "verbose": True,
                },
            },
        },
        "seg_model": "Unet",
    },
    "output_dir": "models",
    "progress_bar_refresh_rate": 10,
    "seed": 42,
    "train_bs": 1,
    "use_aug": True,
    "trainer": {
        "enable_progress_bar": True,
        "max_epochs": 20,
        "min_epochs": 12,
        "accelerator": "mps",
        "devices": 1,
    },
    "valid_bs": 1,
    "workers": 0,
    "device": "mps",
    "folds": {
        "n_splits": 4,
        "random_state": 42,
        "train_folds": [0, 1, 2, 3]
    },
    'target_col': [ "bowel_injury", 
                   "extravasation_injury", 
                #    "kidney_healthy", 
                   "kidney_low",
                   "kidney_high", 
                #    "liver_healthy", 
                   "liver_low", "liver_high",
                #    "spleen_healthy", 
                   "spleen_low", "spleen_high"],
}


In [2]:
BASE_PATH = '.'

# val
df = pd.read_csv(f"{BASE_PATH}/train.csv")
df["image_path"] = f"{BASE_PATH}/train_images"\
                    + "/" + df.patient_id.astype(str)\
                    + "/" + df.series_id.astype(str)\
                    + "/" + df.instance_number.astype(str) +".png"
df = df.drop_duplicates()

df.head(2)

,patient_id,bowel_healthy,bowel_injury,extravasation_healthy,extravasation_injury,kidney_healthy,kidney_low,kidney_high,liver_healthy,liver_low,...,spleen_healthy,spleen_low,spleen_high,any_injury,series_id,instance_number,injury_name,image_path,width,height
0,10004,1,0,0,1,0,1,0,1,0,...,0,0,1,1,21057,362,Active_Extravasation,./train_images/10004/21057/362.png,512,512
1,10004,1,0,0,1,0,1,0,1,0,...,0,0,1,1,21057,363,Active_Extravasation,./train_images/10004/21057/363.png,512,512


In [3]:
# count of each patient ID in df
df.patient_id.value_counts().max()

410

In [4]:
from collections import defaultdict

class CTDataSet(Dataset):
    def __init__(self, df, labels, transform=None):
        self.labels = labels
        self.transform = transform

        # Group image paths by patient_id
        self.bags = defaultdict(list)
        for idx, row in df.iterrows():
            patient_id = row["patient_id"]
            self.bags[patient_id].append(row["image_path"])

        # Convert to lists for indexing
        self.patient_ids = list(self.bags.keys())
        self.bag_labels = df.drop_duplicates(subset=["patient_id"]).set_index("patient_id")[labels].to_dict(orient="index")

    def __len__(self):
        return len(self.patient_ids)

    def __getitem__(self, idx):
        patient_id = self.patient_ids[idx]
        image_paths = self.bags[patient_id]
        image_paths.sort()
        images = [Image.open(image_path).convert('RGB') for image_path in image_paths]

        grouped_images = []
        for i in range(0, len(images), 5): # try 6
            subset = images[i:min(i+5, len(images))]
            # Convert images in subset to tensors, sum them, and convert back to PIL Image
            summed_image = transforms.ToPILImage()(sum([transforms.ToTensor()(img) for img in subset]))
            grouped_images.append(summed_image)

        if len(grouped_images) > 60:
            grouped_images = grouped_images[::2]

        if self.transform:
            grouped_images = [self.transform(image) for image in grouped_images]


        images_tensor = torch.stack(grouped_images)
        label_data = self.bag_labels[patient_id]
        label = torch.tensor([label_data[label] for label in self.labels], dtype=torch.float32)

        # Splitting the label tensor based on your provided indices
        label_splits = (label[0:1], label[1:2], label[2:3], label[3:4], label[4:5], label[5:6], label[6:7], label[7:8])
        return images_tensor, label_splits

# Define transformations and augmentations
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    # transforms.RandomHorizontalFlip(),
    # transforms.RandomVerticalFlip(),
    transforms.RandomAutocontrast(),
    transforms.RandomRotation(5),
    # transforms.Pad(padding = 20),
    transforms.ColorJitter(),
    # transforms.RandomPerspective(),
    transforms.RandomAffine(degrees=(5, 10), translate=(0.05, 0.1), scale=(0.8, 0.95)),
    transforms.ToTensor(),
    # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
# paths  = train_data.image_path.tolist()
# labels = train_data[config.TARGET_COLS].values

* All the images **should** be the same orientation.
* Applying flips might weaken the model as it can reasonably expect a certain orientation
* some people have situs inversus (rare)
* if the trauma is bad the person might not by lying straight, so there may be a clue in their having trauma from the angle of the image
* applying radom rotations might destroy this information

In [5]:
class MetricsCalculator:
    
    def __init__(self, mode = 'binary'):
        
        self.probabilities = []
        self.predictions = []
        self.targets = []
        
        self.mode = mode
    
    def update(self, logits, target):
        """
        Update the metrics calculator with predicted values and corresponding targets.
        
        Args:
            predicted (torch.Tensor): Predicted values.
            target (torch.Tensor): Ground truth targets.
        """
        if self.mode == 'binary':
            probabilities = torch.sigmoid(logits)
            predicted = (probabilities > 0.5)
        else:
            probabilities = F.softmax(logits, dim = 1)
            predicted = torch.argmax(probabilities, dim=1)
            
        self.probabilities.extend(probabilities.detach().cpu().numpy())
        self.predictions.extend(predicted.detach().cpu().numpy())
        self.targets.extend(target.detach().cpu().numpy())
    
    def reset(self):
        """Reset the stored predictions and targets."""
        
        self.probabilities = []
        self.predictions = []
        self.targets = []
    
    def compute_accuracy(self):
        """
        Compute the accuracy metric.
        
        Returns:
            float: Accuracy.
        """
        return accuracy_score(self.targets, self.predictions)
    
    def compute_auc(self):
        """
        Compute the AUC (Area Under the Curve) metric.
        
        Returns:
            float: AUC.
        """
        if self.mode == 'multi':
            return roc_auc_score(self.targets, self.probabilities, multi_class = 'ovo', labels=[0, 1, 2])
    
        else:
            return roc_auc_score(self.targets, self.probabilities)

In [6]:
class LightningModule(pl.LightningModule):
    def __init__(self, config):
        super().__init__()
        self.config = config

        # self.input = nn.Conv2d(3, 3, kernel_size = 3)
        model = self.config['encoder_name']
        
        self.features = model.features
        self.avgpool = model.avgpool
        
        self.bowel = nn.Linear(1280, 1)
        self.extravasation = nn.Linear(1280, 1)
        self.kidney_low = nn.Linear(1280, 1)
        self.kidney_high = nn.Linear(1280, 1)
        self.liver_low = nn.Linear(1280,1) 
        self.liver_high = nn.Linear(1280, 1)
        self.spleen_low = nn.Linear(1280, 1)
        self.spleen_high = nn.Linear(1280, 1)

        self.bowel_metrics = MetricsCalculator(mode='binary')
        self.extravasation_metrics = MetricsCalculator(mode='binary')
        self.kidney_low_metrics = MetricsCalculator(mode='binary')
        self.kidney_high_metrics = MetricsCalculator(mode='binary')
        self.liver_low_metrics = MetricsCalculator(mode='binary')
        self.liver_high_metrics = MetricsCalculator(mode='binary')
        self.spleen_low_metrics = MetricsCalculator(mode='binary')
        self.spleen_high_metrics = MetricsCalculator(mode='binary')
        
        self.loss_module = nn.BCEWithLogitsLoss()
        self.val_step_outputs = []
        self.val_step_labels = []
        self.save_hyperparameters()

    def forward(self, bags):
        # Each bag contains multiple instances (images)
        bag_level_preds = []
        for bag in bags:
            x = self.features(bag)  # Process each image through the feature extractor
            x = self.avgpool(x)
            x = torch.flatten(x, 1)

            # Output logits for each task
            bowel = self.bowel(x)
            extravasation = self.extravasation(x)
            kidney_low = self.kidney_low(x)
            kidney_high = self.kidney_high(x)
            liver_low = self.liver_low(x)
            liver_high = self.liver_high(x)
            spleen_low = self.spleen_low(x)
            spleen_high = self.spleen_high(x)


            # Aggregate instance-level predictions to bag-level predictions
            bag_level_preds.append(torch.stack([bowel, extravasation, kidney_low, kidney_high, liver_low, liver_high, spleen_low, spleen_high], dim=0).max(dim=1)[0])

        return torch.stack(bag_level_preds, dim=0).squeeze(2)

    def configure_optimizers(self):
        optimizer = AdamW(self.parameters(), **self.config["optimizer_params"])

        if self.config["scheduler"]["name"] == "CosineAnnealingLR":
            scheduler = CosineAnnealingLR(
                optimizer,
                **self.config["scheduler"]["params"][self.config["scheduler"]["name"]],
            )
            lr_scheduler_dict = {"scheduler": scheduler, "interval": "step"}
            return {"optimizer": optimizer, "lr_scheduler": lr_scheduler_dict}
        elif self.config["scheduler"]["name"] == "ReduceLROnPlateau":
            scheduler = ReduceLROnPlateau(
                optimizer,
                **self.config["scheduler"]["params"][self.config["scheduler"]["name"]],
            )
            lr_scheduler = {"scheduler": scheduler, "monitor": "val_loss"}
            return {"optimizer": optimizer, "lr_scheduler": lr_scheduler}

    def training_step(self, batch, batch_idx):
        bags, (bowel_labels, extravasation_labels, kidney_low_labels, kidney_high_labels, liver_low_labels, liver_high_labels, spleen_low_labels, spleen_high_labels) = batch
        bag_level_preds = self(bags)

        # Combine all the labels and predictions into single tensors
        all_labels = torch.cat((bowel_labels, extravasation_labels, kidney_low_labels, kidney_high_labels, liver_low_labels, liver_high_labels, spleen_low_labels, spleen_high_labels), dim=1)
        
        # Compute loss
        loss = self.loss_module(bag_level_preds, all_labels.float())
        self.log("train_loss", loss, on_step=True, on_epoch=True, prog_bar=True, batch_size=16)
        for param_group in self.trainer.optimizers[0].param_groups:
            lr = param_group["lr"]
        self.log("lr", lr, on_step=True, on_epoch=True, prog_bar=True)

        # self.bowel_metrics.update(bag_level_preds[:, 0], bowel_labels)
        # self.extravasation_metrics.update(bag_level_preds[:, 1], extravasation_labels)
        # self.kidney_low_metrics.update(bag_level_preds[:, 2], kidney_low_labels)
        # self.kidney_high_metrics.update(bag_level_preds[:, 3], kidney_high_labels)
        # self.liver_low_metrics.update(bag_level_preds[:, 4], liver_low_labels)
        # self.liver_high_metrics.update(bag_level_preds[:, 5], liver_high_labels)
        # self.spleen_low_metrics.update(bag_level_preds[:, 6], spleen_low_labels)
        # self.spleen_high_metrics.update(bag_level_preds[:, 7], spleen_high_labels)

        return loss

    def validation_step(self, batch, batch_idx):
        bags, (bowel_labels, extravasation_labels, kidney_low_labels, kidney_high_labels, liver_low_labels, liver_high_labels, spleen_low_labels, spleen_high_labels) = batch
        bag_level_preds = self(bags)
        
        # Combine all the labels and predictions into single tensors
        all_labels = torch.cat((bowel_labels, extravasation_labels, kidney_low_labels, kidney_high_labels, liver_low_labels, liver_high_labels, spleen_low_labels, spleen_high_labels), dim=1)
        
        # Compute loss
        loss = self.loss_module(bag_level_preds, all_labels.float())
        
        self.log("val_loss", loss, on_step=False, on_epoch=True, prog_bar=True)

        self.bowel_metrics.update(bag_level_preds[:, 0], bowel_labels)
        self.extravasation_metrics.update(bag_level_preds[:, 1], extravasation_labels)
        self.kidney_low_metrics.update(bag_level_preds[:, 2], kidney_low_labels)
        self.kidney_high_metrics.update(bag_level_preds[:, 3], kidney_high_labels)
        self.liver_low_metrics.update(bag_level_preds[:, 4], liver_low_labels)
        self.liver_high_metrics.update(bag_level_preds[:, 5], liver_high_labels)
        self.spleen_low_metrics.update(bag_level_preds[:, 6], spleen_low_labels)
        self.spleen_high_metrics.update(bag_level_preds[:, 7], spleen_high_labels)


    def on_validation_epoch_end(self, outputs):
        # Compute and log metrics
        self.log('val_bowel_accuracy', self.bowel_metrics.compute_accuracy(),on_epoch=True)
        self.log('val_bowel_auc', self.bowel_metrics.compute_auc(),on_epoch=True)
        self.log('val_extravasation_accuracy', self.extravasation_metrics.compute_accuracy(),on_epoch=True)
        self.log('val_extravasation_auc', self.extravasation_metrics.compute_auc(),on_epoch=True)
        self.log('val_kidney_low_accuracy', self.kidney_low_metrics.compute_accuracy(),on_epoch=True)
        self.log('val_kidney_low_auc', self.kidney_low_metrics.compute_auc(),on_epoch=True)
        self.log('val_kidney_high_accuracy', self.kidney_high_metrics.compute_accuracy(),on_epoch=True)
        self.log('val_kidney_high_auc', self.kidney_high_metrics.compute_auc(),on_epoch=True)
        self.log('val_liver_low_accuracy', self.liver_low_metrics.compute_accuracy(),on_epoch=True)
        self.log('val_liver_low_auc', self.liver_low_metrics.compute_auc(),on_epoch=True)
        self.log('val_liver_high_accuracy', self.liver_high_metrics.compute_accuracy(),on_epoch=True)
        self.log('val_liver_high_auc', self.liver_high_metrics.compute_auc(),on_epoch=True)
        self.log('val_spleen_low_accuracy', self.spleen_low_metrics.compute_accuracy(),on_epoch=True)
        self.log('val_spleen_low_auc', self.spleen_low_metrics.compute_auc(),on_epoch=True)
        self.log('val_spleen_high_accuracy', self.spleen_high_metrics.compute_accuracy(),on_epoch=True)
        self.log('val_spleen_high_auc', self.spleen_high_metrics.compute_auc(),on_epoch=True)

        # Reset metrics
        self.bowel_metrics.reset()
        self.extravasation_metrics.reset()
        self.kidney_low_metrics.reset()
        self.kidney_high_metrics.reset()
        self.liver_low_metrics.reset()
        self.liver_high_metrics.reset()
        self.spleen_low_metrics.reset()
        self.spleen_high_metrics.reset()

In [7]:
import warnings
import gc

warnings.filterwarnings("ignore")

import os
import torch
import pandas as pd
import lightning.pytorch as pl
from pprint import pprint
from lightning.pytorch.callbacks import ModelCheckpoint, EarlyStopping, TQDMProgressBar
from torch.utils.data import DataLoader
from sklearn.model_selection import KFold
from torch import mps
from sklearn.model_selection import StratifiedGroupKFold

df['stratify'] = ''
for col in config['target_col']:
    df['stratify'] += df[col].astype(str)

df = df.reset_index(drop=True)
skf = StratifiedGroupKFold(n_splits=config['folds']['n_splits'], shuffle=True, random_state=config['seed'])
for fold, (train_idx, val_idx) in enumerate(skf.split(df, df['stratify'], df["patient_id"])):
    df.loc[val_idx, 'fold'] = fold

    train_df = df.iloc[train_idx]
    valid_df = df.iloc[val_idx]

    train_df = train_df.reset_index(drop=True)
    valid_df = valid_df.reset_index(drop=True)

    dataset_train = CTDataSet(train_df, labels=config['target_col'], transform=transform)
    dataset_validation = CTDataSet(valid_df, labels=config['target_col'], transform=transform)

    data_loader_train = DataLoader(
        dataset_train, batch_size=config["train_bs"], shuffle=True, num_workers=config["workers"]
    )
    data_loader_validation = DataLoader(
        dataset_validation, batch_size=config["valid_bs"], shuffle=False, num_workers=config["workers"]
    )

    pl.seed_everything(config["seed"])

    filename = f"model_fold_{fold}"

    checkpoint_callback = ModelCheckpoint(
        monitor="val_loss",
        dirpath=config["output_dir"],
        mode="min",
        filename=filename,
        save_top_k=1,
        verbose=1,
    )

    progress_bar_callback = TQDMProgressBar(refresh_rate=config["progress_bar_refresh_rate"])

    early_stop_callback = EarlyStopping(monitor="val_loss", mode="min", patience=5, verbose=1)

    wandb_logger = WandbLogger(log_model="all")

    trainer = pl.Trainer(
        callbacks=[checkpoint_callback, early_stop_callback, progress_bar_callback], logger=wandb_logger, **config["trainer"]
    )

    model = LightningModule(config['model'])

    trainer.fit(model, data_loader_train, data_loader_validation)
    torch.save(model.state_dict(), os.path.join(config["output_dir"], f"{filename}.pth"))
    model = model.to('cpu')
    mps.empty_cache()
    del (
        dataset_train,
        dataset_validation,
        train_df,
        valid_df,
        data_loader_train,
        data_loader_validation,
        model,
        trainer,
        checkpoint_callback,
        progress_bar_callback,
        early_stop_callback,
    )
    mps.empty_cache()
    gc.collect()

Global seed set to 42
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: johnny-hyland. Use `wandb login --relogin` to force relogin


GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

   | Name          | Type              | Params
-----------------------------------------------------
0  | features      | Sequential        | 6.5 M 
1  | avgpool       | AdaptiveAvgPool2d | 0     
2  | bowel         | Linear            | 1.3 K 
3  | extravasation | Linear            | 1.3 K 
4  | kidney_low    | Linear            | 1.3 K 
5  | kidney_high   | Linear            | 1.3 K 
6  | liver_low     | Linear            | 1.3 K 
7  | liver_high    | Linear            | 1.3 K 
8  | spleen_low    | Linear            | 1.3 K 
9  | spleen_high   | Linear            | 1.3 K 
10 | loss_module   | BCEWithLogitsLoss | 0     
-----------------------------------------------------
6.5 M     Trainable params
0         Non-trainable params
6.5 M     Total params
26.094    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Monitored metric val_loss did not improve in the last 7 records. Best score: 0.377. Signaling Trainer to stop.
Epoch 9, global step 1800: 'val_loss' was not in top 1


Validation: 0it [00:00, ?it/s]

Monitored metric val_loss did not improve in the last 8 records. Best score: 0.377. Signaling Trainer to stop.
Epoch 10, global step 1980: 'val_loss' was not in top 1


Validation: 0it [00:00, ?it/s]

Monitored metric val_loss did not improve in the last 9 records. Best score: 0.377. Signaling Trainer to stop.
Epoch 11, global step 2160: 'val_loss' was not in top 1
